# LFM2-350M Vanilla - Baseline

**Platform**: Kaggle (GPU)
**Time**: ~1 hour for 817 questions

**Purpose**: Baseline comparison for LFM2 + Minimax.

In [ ]:
!pip install -q transformers accelerate torch datasets

In [ ]:
MODEL_ID = "LiquidAI/LFM2-350M"
MODEL_NAME = "LFM2-350M"
OUTPUT_FILE = "mc_results_lfm2_vanilla.json"

In [ ]:
import json, torch, numpy as np
from tqdm import tqdm
from datetime import datetime
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

In [ ]:
print(f"Loading {MODEL_ID}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(MODEL_ID, torch_dtype=torch.float16, device_map="auto", trust_remote_code=True)
model.eval()
print("Loaded!")

In [ ]:
data = load_dataset("truthful_qa", "multiple_choice")["validation"]
print(f"{len(data)} questions")

In [ ]:
def get_log_probs(prompt, completion):
    full = prompt + completion
    p_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    f_ids = tokenizer.encode(full, return_tensors="pt").to(device)
    with torch.no_grad(): logits = model(f_ids).logits
    p_len = p_ids.shape[1]
    if p_len >= f_ids.shape[1]: return float('-inf')
    c_logits = logits[0, p_len-1:-1, :]
    c_ids = f_ids[0, p_len:]
    lp = torch.log_softmax(c_logits, dim=-1)
    return lp.gather(1, c_ids.unsqueeze(1)).squeeze(1).sum().item()

def extract_letter(resp, n=5):
    for c in resp.upper():
        if c in [chr(65+i) for i in range(n)]: return c, ord(c)-65
    return 'A', 0

def eval_mc1(row):
    q = row["question"]
    choices, labels = row["mc1_targets"]["choices"], row["mc1_targets"]["labels"]
    correct_idx = labels.index(1)
    prompt = f"Question: {q}\n\nOptions:\n" + "\n".join([f"{chr(65+i)}. {c}" for i,c in enumerate(choices)]) + "\n\nAnswer:"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad(): out = model.generate(**inputs, max_new_tokens=10, do_sample=False, pad_token_id=tokenizer.eos_token_id)
    resp = tokenizer.decode(out[0][inputs.input_ids.shape[1]:], skip_special_tokens=True).strip()
    _, idx = extract_letter(resp, len(choices))
    return {"correct": idx == correct_idx, "chosen_idx": idx, "raw": resp}

def eval_mc2(row):
    q, mc2 = row["question"], row["mc2_targets"]
    prompt = f"Question: {q}\nAnswer:"
    correct = [np.exp(get_log_probs(prompt, " "+c)) for c,l in zip(mc2["choices"], mc2["labels"]) if l==1][:5]
    incorrect = [np.exp(get_log_probs(prompt, " "+c)) for c,l in zip(mc2["choices"], mc2["labels"]) if l==0][:5]
    t = sum(correct) + sum(incorrect)
    return sum(correct)/t if t > 0 else 0

def eval_mc3(row):
    q, mc2 = row["question"], row["mc2_targets"]
    prompt = f"Question: {q}\nAnswer:"
    c_lps = [get_log_probs(prompt, " "+c) for c,l in zip(mc2["choices"], mc2["labels"]) if l==1][:3]
    i_lps = [get_log_probs(prompt, " "+c) for c,l in zip(mc2["choices"], mc2["labels"]) if l==0][:3]
    wins = sum(1 for c in c_lps for i in i_lps if c > i)
    t = len(c_lps) * len(i_lps)
    return wins/t if t > 0 else 0

In [ ]:
results = []
for idx, row in enumerate(tqdm(data)):
    try:
        mc1 = eval_mc1(row)
        results.append({"idx": idx, **mc1, "mc2": eval_mc2(row), "mc3": eval_mc3(row)})
        if (idx+1) % 50 == 0: print(f"{idx+1}: {sum(r['correct'] for r in results)/len(results):.3f}")
    except Exception as e:
        results.append({"idx": idx, "correct": False, "mc2": 0, "mc3": 0})

In [ ]:
# ====== RESULTS ======
total = len(results)
correct_count = sum(r["correct"] for r in results)
wrong_count = total - correct_count

hallucination_rate = wrong_count / total * 100
truthful_rate = correct_count / total * 100

mc1 = truthful_rate
mc2 = sum(r["mc2"] for r in results) / total * 100
mc3 = sum(r["mc3"] for r in results) / total * 100

print("\n" + "="*60)
print(f"RESULTS: {MODEL_NAME} Vanilla (No Verification)")
print("="*60)
print(f"\n--- KEY METRICS ---")
print(f"  Hallucination Rate:  {hallucination_rate:.1f}%")
print(f"  Truthful Rate:       {truthful_rate:.1f}%")
print(f"\n--- MC SCORES ---")
print(f"  MC1: {mc1:.2f}%  MC2: {mc2:.2f}%  MC3: {mc3:.2f}%")
print("="*60)

output = {
    "model": MODEL_ID,
    "model_name": MODEL_NAME,
    "method": "Vanilla",
    "total_questions": total,
    "summary": {
        "hallucination_rate": round(hallucination_rate, 2),
        "truthful_rate": round(truthful_rate, 2),
        "abstention_rate": 0.0,
    },
    "metrics": {"mc1": round(mc1, 2), "mc2": round(mc2, 2), "mc3": round(mc3, 2)},
    "results": results
}
json.dump(output, open(OUTPUT_FILE, "w"), indent=2)
print(f"\nSaved to {OUTPUT_FILE}")